In [1]:
# You can use python inline. E.g. you can define functions to be used later:

# import environment and expose in jupyter env
# this only works when shellvars is installed as it is in L-CAS Juptyer images
# https://github.com/LCAS/jupyterhub-deploy-docker/blob/master/singleuser/Dockerfile#L100

import shellvars
from os import environ

def loadenv(filename='/opt/ros/kinetic/setup.bash'):
    vs = shellvars.get_vars(filename)
    for v in vs:
        print('%s=%s' % (v.decode("utf-8"), str(vs[v].decode("utf-8"))))
        environ[v.decode("utf-8")] = str(vs[v].decode("utf-8"))

# you can call these functions easily
# (This example is a good way to load environment variables from a config file)

loadenv('/opt/ros/kinetic/setup.bash')

ROS_ROOT=/opt/ros/kinetic/share/ros
ROS_PACKAGE_PATH=/opt/ros/kinetic/share
ROS_MASTER_URI=http://localhost:11311
ROS_VERSION=1
LD_LIBRARY_PATH=/opt/ros/kinetic/lib:/opt/ros/kinetic/lib/x86_64-linux-gnu
PATH=/opt/ros/kinetic/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
ROS_DISTRO=kinetic
PYTHONPATH=/opt/ros/kinetic/lib/python2.7/dist-packages
PKG_CONFIG_PATH=/opt/ros/kinetic/lib/pkgconfig:/opt/ros/kinetic/lib/x86_64-linux-gnu/pkgconfig
CMAKE_PREFIX_PATH=/opt/ros/kinetic
ROS_ETC_DIR=/opt/ros/kinetic/etc/ros


In [2]:
# install marc-hanheide's own python3 version of rosbag_panda: https://github.com/marc-hanheide/RosbagPandas
#   - clone it
#   - run pip install --user -U . in it 
# tutorial at https://nimbus.unl.edu/2014/11/using-rosbag_pandas-to-analyze-rosbag-files/
#sudo apt-get install ros-kinetic-rosbag python-roslz4
def expand_sys_path():
    import sys
    from os import environ

    extra_paths = environ['PYTHONPATH'].split(':')
    sys.path.extend(extra_paths)

    new_path = []
    for p in sys.path:
        if not p in new_path:
            new_path.append(p)

    sys.path = new_path


expand_sys_path()

import rosbag
import pixiedust
from metakernel.display import display
from pprint import pprint


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


Pixiedust database opened successfully


In [66]:
import pandas as pd
import numpy as np

def bag2df(bagfile):

    bag = rosbag.Bag(bagfile)
    topics = bag.get_type_and_topic_info()[1]

    for t in topics:
        print(t, topics[t])

    import rosbag_pandas
    dfs = {
       'qtc': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/online_qtc_creator/qtc_array']),
       'people_pose': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/human_perception/tracked_persons']),
       'robot_pose': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/robot_poseST']),
    }
    dfs['qtc']['ts'] = (
        dfs['qtc']['/robot5/online_qtc_creator/qtc_array/header/stamp/secs'] +
        (dfs['qtc']['/robot5/online_qtc_creator/qtc_array/header/stamp/nsecs'] / 10**9)
    )
    dfs['people_pose']['ts'] = (
        dfs['people_pose']['/robot5/human_perception/tracked_persons/header/stamp/secs'] +
        (dfs['people_pose']['/robot5/human_perception/tracked_persons/header/stamp/nsecs'] / 10**9)
    )
    dfs['robot_pose']['ts'] = (
        dfs['robot_pose']['/robot5/robot_poseST/header/stamp/secs'] +
        (dfs['robot_pose']['/robot5/robot_poseST/header/stamp/nsecs'] / 10**9)
    )

    #for d in dfs:
    #    df[d].set_index('ts', drop=False, inplace=True)
    #    df[d].sort_index(inplace=True)

    
#    df = rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/online_qtc_creator/qtc_array'])
    return dfs
    
df = bag2df('bags/S1-T1.1-A1_tiny_2019-02-05-19-31-12.bag')


/robot5/human_perception/tracked_persons TopicTuple(msg_type='spencer_tracking_msgs/TrackedPersons', message_count=36, connections=1, frequency=None)
/robot5/online_qtc_creator/qtc_array TopicTuple(msg_type='hrsi_representation/QTCArray', message_count=5, connections=1, frequency=0.6393609902212601)
/robot5/people_tracker/positions TopicTuple(msg_type='bayes_people_tracker/PeopleTracker', message_count=36, connections=1, frequency=None)
/robot5/robot_poseST TopicTuple(msg_type='geometry_msgs/PoseStamped', message_count=102, connections=1, frequency=9.92445749521441)
/tf TopicTuple(msg_type='tf2_msgs/TFMessage', message_count=5754, connections=5, frequency=None)
/tf_static TopicTuple(msg_type='tf2_msgs/TFMessage', message_count=2, connections=2, frequency=1.0900162373100537)


In [76]:
from yaml import load
#df['people_pose'].info()
#display(df['people_pose'])

df_qtc_people = pd.merge_asof(
    df['people_pose'], df['qtc'])
df_all = pd.merge_asof(
    df_qtc_people, df['robot_pose'])

#display(df_all)
#pprint(df_all.columns.tolist())

start_time=min(df_all['ts'])
df_all['time'] = df_all['ts'] - start_time


df_selected = df_all[
    [
        'time',
        'ts',
        '/robot5/online_qtc_creator/qtc_array/qtc/0/qtc_robot_human',
        '/robot5/human_perception/tracked_persons/tracks/0/pose/pose/position/x',
        '/robot5/human_perception/tracked_persons/tracks/0/pose/pose/position/y',
        '/robot5/robot_poseST/pose/position/x',
        '/robot5/robot_poseST/pose/position/y']
]
df_selected.dropna(inplace=True)

display(df_selected)





/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,time,ts,/robot5/online_qtc_creator/qtc_array/qtc/0/qtc_robot_human,/robot5/human_perception/tracked_persons/tracks/0/pose/pose/position/x,/robot5/human_perception/tracked_persons/tracks/0/pose/pose/position/y,/robot5/robot_poseST/pose/position/x,/robot5/robot_poseST/pose/position/y
28,5.437998,1.544625e+09,"[""+,-,+,+""]",0.114480,-23.265967,-19.408024,-40.379599
29,6.026870,1.544625e+09,"[""+,-,+,+""]",0.066264,-23.246599,-19.408024,-40.379599
30,7.464966,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.067845,-23.233645,-18.479835,-40.754825
31,7.784698,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.128594,-23.243018,-18.370623,-40.792810
32,8.144221,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.116902,-23.233608,-18.370623,-40.792810
33,8.723233,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.101631,-23.226837,-17.978880,-40.918434
34,8.926234,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.135862,-23.237995,-17.978880,-40.918434
35,12.263090,1.544625e+09,"[""+,-,+,+"", ""0,-,+,+"", ""-,-,+,+""]",0.075514,-23.290969,-17.587323,-41.045228
